In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re
import sklearn
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
import random
import typhon
from typhon.retrieval.qrnn import QRNN
import glob

/home/teodor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def pre_process_training_data(input_arr, output_arr):
    # Remove artefacts from data
    for i in range(0,13):
        delete_inds = np.where(input_arr[:,i] > 500)
        input_arr = np.delete(input_arr, (delete_inds), axis = 0)
        output_arr = np.delete(output_arr, (delete_inds), axis = 0)
    
    for i in range(0,input_arr.shape[1]):
        delete_inds = np.where(input_arr[:,i] < 0)
        input_arr = np.delete(input_arr, (delete_inds), axis = 0)
        output_arr = np.delete(output_arr, (delete_inds), axis = 0)
    # Rounding the TCWV and T2m
    input_arr[:,13] = np.around(input_arr[:,13])
    input_arr[:,14] = np.around(input_arr[:,14])
    
    x_train = input_arr
    y_train = output_arr
    y_train = np.reshape(y_train,(y_train.shape[0],1))
    in_dim = x_train.shape[1]
    return x_train, y_train, in_dim

In [3]:
training_input_path = '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input' + '/*.txt'
training_output_path = '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/output' + '/*.txt'

input_list = np.sort(glob.glob(training_input_path))
output_list = np.sort(glob.glob(training_output_path))

In [4]:
print(input_list)

['/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_1.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_10.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_11.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_12.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_13.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_14.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_15.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_16.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_17.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_18.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/training_data/input/training_input_19.txt'
 '/home/teodor/Dendrite/UserAreas/Teo/qprof/

In [6]:
# LOADING THE FIRST TRAINING BATCH
file_nbr = 0
input_file = input_list[file_nbr]
output_file = output_list[file_nbr]
input_arr = np.loadtxt(input_file)
output_arr = np.loadtxt(output_file)
x_train_full, y_train_full, in_dim = pre_process_training_data(input_arr,output_arr)
print(x_train_full.shape)

(392628, 15)


In [7]:
# FOR LOADING ALL TRAINING DATA
for i in range(1,len(input_list)):
    input_file = input_list[i]
    output_file = output_list[i]
    input_arr = np.loadtxt(input_file)
    output_arr = np.loadtxt(output_file)
    x_train, y_train, in_dim = pre_process_training_data(input_arr,output_arr)
    x_train_full = np.concatenate((x_train_full,x_train), axis = 0)
    y_train_full = np.concatenate((y_train_full,y_train), axis = 0)
    if np.mod(i,3) == 0:
        print(x_train_full.shape)

(2419691, 15)
(4473965, 15)
(6501882, 15)


In [11]:
print(np.max(y_train_full))
print(len(y_train_full[y_train_full > 20]))
print(x_train_full.shape)

101.50733947753906
2096
(8130520, 15)


In [ ]:
max_ep = 300
quantiles = [0.01,0.1,0.2,0.33,0.4,0.5,0.66,0.7,0.8,0.9,0.99]
in_dim = x_train_full.shape[1]
ensemble_size = 1

depths = [12]
widths = [128,256]

for i in range(0,len(depths)):
    for j in range(0,len(widths)):
        NN_depth = depths[i]
        NN_width = widths[j]
        print(NN_depth)
        print(NN_width)

        qrnn_1 = QRNN(in_dim, quantiles, NN_depth, NN_width, ensemble_size = ensemble_size)
        qrnn_1.fit(x_train_full, y_train_full, sigma_noise=None,
                         adversarival_training = True,
                         delta_at = 0.05,
                         initial_learning_rate = 0.001,
                         learning_rate_decay = 1.5,
                         convergence_epochs = 10,
                         batch_size = 128,
                         maximum_epochs = max_ep,
                         learning_rate_minimum = 1e-4)
        
        qrnn_name = "qrnn_" + str(NN_depth) + '_' + str(NN_width) + '_extra_data'
        qrnn_1.save(qrnn_name)

12
128
Epoch 1/300
11050/57167 [====>.........................] - ETA: 1:53 - loss: 0.3748

In [13]:
# TEST DIFFERENT DELTA AT
max_ep = 300
quantiles = [0.01,0.1,0.2,0.33,0.4,0.5,0.66,0.7,0.8,0.9,0.99]
in_dim = x_train_full.shape[1]
ensemble_size = 1

delta_ats = [0.01, 0.1, 0.2]
at_name = ['001','01','02']

for i in range(0,len(delta_ats)):
    delta_at = delta_ats[i]
    NN_depth = 12
    NN_width = 128
    print(NN_depth)
    print(NN_width)
    print(delta_at)

    qrnn_1 = QRNN(in_dim, quantiles, NN_depth, NN_width, ensemble_size = ensemble_size)
    qrnn_1.fit(x_train_full, y_train_full, sigma_noise=None,
                     adversarival_training = True,
                     delta_at = delta_at,
                     initial_learning_rate = 0.001,
                     learning_rate_decay = 1.5,
                     convergence_epochs = 10,
                     batch_size = 128,
                     maximum_epochs = max_ep,
                     learning_rate_minimum = 1e-4)

    qrnn_name = "qrnn_" + str(NN_depth) + '_' + str(NN_width) + '_at_' + at_name[i]
    qrnn_1.save(qrnn_name)

12
128
0.01
Epoch 1/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.2793 - val_loss: 0.1876
Epoch 2/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1808 - val_loss: 0.1787
Epoch 3/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1742 - val_loss: 0.1710
Epoch 4/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1710 - val_loss: 0.1684
Epoch 5/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1688 - val_loss: 0.1700
Epoch 6/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1673 - val_loss: 0.1660
Epoch 7/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1659 - val_loss: 0.1679
Epoch 8/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1646 - val_loss: 0.1654
Epoch 9/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1636 - val_loss: 0.1616
Epoch 10/300
31457/31457 

Epoch 77/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1478 - val_loss: 0.1483
Epoch 78/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1478 - val_loss: 0.1476
Epoch 79/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1478 - val_loss: 0.1479
Epoch 80/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1477 - val_loss: 0.1490
Epoch 81/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1476 - val_loss: 0.1484
Epoch 82/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1475 - val_loss: 0.1479
Epoch 83/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1475 - val_loss: 0.1494
Epoch 84/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1474 - val_loss: 0.1491
Epoch 85/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1474 - val_loss: 0.1475
Epoch 86/300
31457/31457 [==

31457/31457 [==============================] - 106s 3ms/step - loss: 0.1433 - val_loss: 0.1455
Epoch 153/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1434 - val_loss: 0.1451
Epoch 154/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1434 - val_loss: 0.1455
Epoch 155/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1433 - val_loss: 0.1455
Epoch 156/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1432 - val_loss: 0.1456
Epoch 157/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1433 - val_loss: 0.1450
Epoch 158/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1432 - val_loss: 0.1455
Epoch 159/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1433 - val_loss: 0.1451
Epoch 160/300
31457/31457 [==============================] - 106s 3ms/step - loss: 0.1432 - val_loss: 0.1450
Epoch 161/300
31457/31457 [======

31457/31457 [==============================] - 108s 3ms/step - loss: 0.1540 - val_loss: 0.1539
Epoch 31/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1538 - val_loss: 0.1543
Epoch 32/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1535 - val_loss: 0.1549
Epoch 33/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1532 - val_loss: 0.1538
Epoch 34/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1531 - val_loss: 0.1529
Epoch 35/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1529 - val_loss: 0.1527
Epoch 36/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1528 - val_loss: 0.1537
Epoch 37/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1525 - val_loss: 0.1523
Epoch 38/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1524 - val_loss: 0.1529
Epoch 39/300
31457/31457 [===============

Epoch 106/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1464 - val_loss: 0.1475
Epoch 107/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1464 - val_loss: 0.1488
Epoch 108/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1463 - val_loss: 0.1484
Epoch 109/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1463 - val_loss: 0.1478
Epoch 110/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1462 - val_loss: 0.1475
Epoch 111/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1462 - val_loss: 0.1474
Epoch 112/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1461 - val_loss: 0.1480
Epoch 113/300
31457/31457 [==============================] - 107s 3ms/step - loss: 0.1461 - val_loss: 0.1471
Epoch 114/300
31457/31457 [==============================] - 108s 3ms/step - loss: 0.1460 - val_loss: 0.1478
Epoch 115/300
31457


 Reduced learning rate to 8.77915e-05
12
128
0.2
Epoch 1/300
31457/31457 [==============================] - 110s 3ms/step - loss: 0.3079 - val_loss: 0.1979
Epoch 2/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1844 - val_loss: 0.1788
Epoch 3/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1764 - val_loss: 0.1748
Epoch 4/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1730 - val_loss: 0.1715
Epoch 5/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1706 - val_loss: 0.1808
Epoch 6/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1687 - val_loss: 0.1679
Epoch 7/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1674 - val_loss: 0.1675
Epoch 8/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1662 - val_loss: 0.1666
Epoch 9/300
31457/31457 [==============================] - 109s 3ms/step - loss: 0.1651 - val_

KeyboardInterrupt: 

In [6]:
# ITERATIVE TRAINING!
max_ep = 300
quantiles = [0.01,0.1,0.2,0.33,0.4,0.5,0.66,0.7,0.8,0.9,0.99]
in_dim = 15
ensemble_size = 1
NN_depth = 12
NN_width = 128
qrnn_1 = QRNN(in_dim, quantiles, NN_depth, NN_width, ensemble_size = ensemble_size)

In [ ]:
max_ep = 5
nbr_full_runs = 30
init_learning_rate = 0.001
save_stages = [5,10,15,20]
for j in range(0,nbr_full_runs):
    print('Starting full training run nbr ' + str(j+1))
    for i in range(0,len(input_list)):
        print('Starting training on batch ' + str(i+1))
        input_file = input_list[i]
        output_file = output_list[i]
        input_arr = np.loadtxt(input_file)
        output_arr = np.loadtxt(output_file)
        x_train, y_train, _ = pre_process_training_data(input_arr,output_arr)
        print('Training set size: ' + str(len(y_train)))
        print('Max rain rate: ' + str(np.max(y_train)))
        qrnn_1.fit(x_train, y_train, sigma_noise=None,
                             adversarival_training = True,
                             delta_at = 0.05,
                             initial_learning_rate = init_learning_rate,
                             learning_rate_decay = 1.5,
                             convergence_epochs = 10,
                             batch_size = 128,
                             maximum_epochs = max_ep,
                             learning_rate_minimum = 1e-4)
        
    init_learning_rate = init_learning_rate / 1.5
    if j in save_stages:
        print('Saving qrnn number ' + str(j+1))
        qrnn_name = "qrnn_" + str(NN_depth) + '_' + str(NN_width) + '_' + 'iteration_' + str(j+1)
        qrnn_1.save(qrnn_name)

qrnn_name = "qrnn_" + str(NN_depth) + '_' + str(NN_width) + '_' + 'final'
qrnn_1.save(qrnn_name)

Starting full training run nbr 1
Starting training on batch 1
Training set size: 392628
Max rain rate: 81.79002380371094
Epoch 1/5
2760/2760 [==============================] - 10s 4ms/step - loss: 0.5932 - val_loss: 0.5687
Epoch 2/5
2760/2760 [==============================] - 9s 3ms/step - loss: 0.5678 - val_loss: 0.5423
Epoch 3/5
2760/2760 [==============================] - 10s 3ms/step - loss: 0.4841 - val_loss: 0.3370
Epoch 4/5
2760/2760 [==============================] - 11s 4ms/step - loss: 0.3023 - val_loss: 0.2683
Epoch 5/5
2760/2760 [==============================] - 11s 4ms/step - loss: 0.2695 - val_loss: 0.2490
Starting training on batch 2
Training set size: 408188
Max rain rate: 101.50733947753906
Epoch 1/5
2870/2870 [==============================] - 10s 4ms/step - loss: 0.2323 - val_loss: 0.2108
Epoch 2/5
2870/2870 [==============================] - 10s 3ms/step - loss: 0.2202 - val_loss: 0.2006
Epoch 3/5
2870/2870 [==============================] - 10s 3ms/step - loss: 0

Epoch 4/5
2786/2786 [==============================] - 10s 4ms/step - loss: 0.1743 - val_loss: 0.1738
Epoch 5/5
2786/2786 [==============================] - 10s 4ms/step - loss: 0.1738 - val_loss: 0.1725
Starting training on batch 15
Training set size: 419011
Max rain rate: 60.561824798583984
Epoch 1/5
2946/2946 [==============================] - 13s 4ms/step - loss: 0.1743 - val_loss: 0.1787
Epoch 2/5
2946/2946 [==============================] - 11s 4ms/step - loss: 0.1736 - val_loss: 0.1732
Epoch 3/5
2946/2946 [==============================] - 11s 4ms/step - loss: 0.1730 - val_loss: 0.1715
Epoch 4/5
2946/2946 [==============================] - 11s 4ms/step - loss: 0.1723 - val_loss: 0.1691
Epoch 5/5
2946/2946 [==============================] - 11s 4ms/step - loss: 0.1718 - val_loss: 0.1711
Starting training on batch 16
Training set size: 405270
Max rain rate: 66.85426330566406
Epoch 1/5
2849/2849 [==============================] - 13s 4ms/step - loss: 0.1663 - val_loss: 0.1694
Epoch

Epoch 2/5
2903/2903 [==============================] - 11s 4ms/step - loss: 0.1638 - val_loss: 0.1666
Epoch 3/5
2903/2903 [==============================] - 12s 4ms/step - loss: 0.1631 - val_loss: 0.1664
Epoch 4/5
2903/2903 [==============================] - 12s 4ms/step - loss: 0.1630 - val_loss: 0.1658
Epoch 5/5
2903/2903 [==============================] - 11s 4ms/step - loss: 0.1623 - val_loss: 0.1661
Starting training on batch 9
Training set size: 409113
Max rain rate: 94.56874084472656
Epoch 1/5
2876/2876 [==============================] - 16s 5ms/step - loss: 0.1688 - val_loss: 0.1631
Epoch 2/5
2876/2876 [==============================] - 11s 4ms/step - loss: 0.1674 - val_loss: 0.1644
Epoch 3/5
2876/2876 [==============================] - 11s 4ms/step - loss: 0.1672 - val_loss: 0.1696
Epoch 4/5
2876/2876 [==============================] - 11s 4ms/step - loss: 0.1669 - val_loss: 0.1628
Epoch 5/5
2876/2876 [==============================] - 11s 4ms/step - loss: 0.1666 - val_loss: 0

2946/2946 [==============================] - 12s 4ms/step - loss: 0.1631 - val_loss: 0.1598
Epoch 4/5
2946/2946 [==============================] - 12s 4ms/step - loss: 0.1624 - val_loss: 0.1591
Epoch 5/5
2946/2946 [==============================] - 12s 4ms/step - loss: 0.1623 - val_loss: 0.1584
Starting training on batch 16
Training set size: 405270
Max rain rate: 66.85426330566406
Epoch 1/5
2849/2849 [==============================] - 22s 8ms/step - loss: 0.1569 - val_loss: 0.1558
Epoch 2/5
2849/2849 [==============================] - 12s 4ms/step - loss: 0.1560 - val_loss: 0.1555
Epoch 3/5
2849/2849 [==============================] - 12s 4ms/step - loss: 0.1558 - val_loss: 0.1544
Epoch 4/5
2849/2849 [==============================] - 12s 4ms/step - loss: 0.1554 - val_loss: 0.1544
Epoch 5/5
2849/2849 [==============================] - 12s 4ms/step - loss: 0.1551 - val_loss: 0.1547
Starting training on batch 17
Training set size: 415127
Max rain rate: 79.00651550292969
Epoch 1/5
2918/2

Epoch 1/5
2876/2876 [==============================] - 27s 9ms/step - loss: 0.1635 - val_loss: 0.1562
Epoch 2/5
2876/2876 [==============================] - 14s 5ms/step - loss: 0.1626 - val_loss: 0.1551
Epoch 3/5
2876/2876 [==============================] - 14s 5ms/step - loss: 0.1621 - val_loss: 0.1554
Epoch 4/5
2876/2876 [==============================] - 14s 5ms/step - loss: 0.1615 - val_loss: 0.1546
Epoch 5/5
2876/2876 [==============================] - 14s 5ms/step - loss: 0.1615 - val_loss: 0.1546
Starting training on batch 10
Training set size: 407574
Max rain rate: 60.28181076049805
Epoch 1/5
2865/2865 [==============================] - 27s 10ms/step - loss: 0.1513 - val_loss: 0.1493
Epoch 2/5
2865/2865 [==============================] - 14s 5ms/step - loss: 0.1502 - val_loss: 0.1497
Epoch 3/5
2865/2865 [==============================] - 14s 5ms/step - loss: 0.1497 - val_loss: 0.1505
Epoch 4/5
2865/2865 [==============================] - 14s 5ms/step - loss: 0.1495 - val_loss:

...

2760/2760 [==============================] - 87s 32ms/step - loss: 0.1527 - val_loss: 0.1397
Epoch 2/5
2760/2760 [==============================] - 23s 9ms/step - loss: 0.1525 - val_loss: 0.1398
Epoch 3/5
2760/2760 [==============================] - 23s 8ms/step - loss: 0.1522 - val_loss: 0.1398
Epoch 4/5
2760/2760 [==============================] - 23s 8ms/step - loss: 0.1523 - val_loss: 0.1394
Epoch 5/5
2760/2760 [==============================] - 24s 9ms/step - loss: 0.1522 - val_loss: 0.1394
Starting training on batch 2
Training set size: 408188
Max rain rate: 101.50733947753906
Epoch 1/5
2870/2870 [==============================] - 82s 29ms/step - loss: 0.1449 - val_loss: 0.1366
Epoch 2/5
2870/2870 [==============================] - 23s 8ms/step - loss: 0.1445 - val_loss: 0.1364
Epoch 3/5
2870/2870 [==============================] - 25s 9ms/step - loss: 0.1443 - val_loss: 0.1364
Epoch 4/5
2870/2870 [==============================] - 23s 8ms/step - loss: 0.1443 - val_loss: 0.1363
E

Epoch 5/5
2786/2786 [==============================] - 23s 8ms/step - loss: 0.1591 - val_loss: 0.1558
Starting training on batch 15
Training set size: 419011
Max rain rate: 60.561824798583984
Epoch 1/5
2946/2946 [==============================] - 93s 32ms/step - loss: 0.1589 - val_loss: 0.1586
Epoch 2/5
2946/2946 [==============================] - 24s 8ms/step - loss: 0.1586 - val_loss: 0.1585
Epoch 3/5
2946/2946 [==============================] - 24s 8ms/step - loss: 0.1584 - val_loss: 0.1584
Epoch 4/5
2946/2946 [==============================] - 24s 8ms/step - loss: 0.1583 - val_loss: 0.1585
Epoch 5/5
2946/2946 [==============================] - 24s 8ms/step - loss: 0.1582 - val_loss: 0.1586
Starting training on batch 16
Training set size: 405270
Max rain rate: 66.85426330566406
Epoch 1/5
2849/2849 [==============================] - 87s 30ms/step - loss: 0.1513 - val_loss: 0.1541
Epoch 2/5
2849/2849 [==============================] - 24s 8ms/step - loss: 0.1510 - val_loss: 0.1542
Epo

2760/2760 [==============================] - 25s 9ms/step - loss: 0.1507 - val_loss: 0.1536
Starting training on batch 2
Training set size: 408188
Max rain rate: 101.50733947753906
Epoch 1/5
2870/2870 [==============================] - 98s 34ms/step - loss: 0.1442 - val_loss: 0.1443
Epoch 2/5
2870/2870 [==============================] - 26s 9ms/step - loss: 0.1436 - val_loss: 0.1441
Epoch 3/5
2870/2870 [==============================] - 26s 9ms/step - loss: 0.1435 - val_loss: 0.1440
Epoch 4/5
2870/2870 [==============================] - 26s 9ms/step - loss: 0.1434 - val_loss: 0.1440
Epoch 5/5
2870/2870 [==============================] - 26s 9ms/step - loss: 0.1434 - val_loss: 0.1440
Starting training on batch 3
Training set size: 407461
Max rain rate: 83.2109603881836
Epoch 1/5
2864/2864 [==============================] - 98s 34ms/step - loss: 0.1515 - val_loss: 0.1516
Epoch 2/5
2864/2864 [==============================] - 26s 9ms/step - loss: 0.1509 - val_loss: 0.1514
Epoch 3/5
2864/2

2903/2903 [==============================] - 33s 12ms/step - loss: 0.1544 - val_loss: 0.1663
Starting training on batch 9
Training set size: 409113
Max rain rate: 94.56874084472656
Epoch 1/5
2876/2876 [==============================] - 116s 40ms/step - loss: 0.1582 - val_loss: 0.1672
Epoch 2/5
2876/2876 [==============================] - 33s 12ms/step - loss: 0.1580 - val_loss: 0.1671
Epoch 3/5
2876/2876 [==============================] - 33s 12ms/step - loss: 0.1579 - val_loss: 0.1671
Epoch 4/5
2876/2876 [==============================] - 33s 12ms/step - loss: 0.1578 - val_loss: 0.1670
Epoch 5/5
2876/2876 [==============================] - 33s 12ms/step - loss: 0.1578 - val_loss: 0.1670
Starting training on batch 10
Training set size: 407574
Max rain rate: 60.28181076049805
Epoch 1/5
2865/2865 [==============================] - 116s 41ms/step - loss: 0.1480 - val_loss: 0.1437
Epoch 2/5
2865/2865 [==============================] - 33s 12ms/step - loss: 0.1478 - val_loss: 0.1436
Epoch 3

2946/2946 [==============================] - 31s 11ms/step - loss: 0.1589 - val_loss: 0.1537
Starting training on batch 16
Training set size: 405270
Max rain rate: 66.85426330566406
Epoch 1/5
2849/2849 [==============================] - 128s 45ms/step - loss: 0.1516 - val_loss: 0.1518
Epoch 2/5
2849/2849 [==============================] - 35s 12ms/step - loss: 0.1514 - val_loss: 0.1517
Epoch 3/5
2849/2849 [==============================] - 35s 12ms/step - loss: 0.1513 - val_loss: 0.1517
Epoch 4/5
2849/2849 [==============================] - 35s 12ms/step - loss: 0.1513 - val_loss: 0.1516
Epoch 5/5
2849/2849 [==============================] - 35s 12ms/step - loss: 0.1512 - val_loss: 0.1516
Starting training on batch 17
Training set size: 415127
Max rain rate: 79.00651550292969
Epoch 1/5
2918/2918 [==============================] - 129s 44ms/step - loss: 0.1525 - val_loss: 0.1525
Epoch 2/5
2918/2918 [==============================] - 36s 12ms/step - loss: 0.1524 - val_loss: 0.1524
Epoch 